# Grafové databáze
Grafové databáze pro reprezentaci dat nepoužívají tabulky, nýbrž nody (vrcholy) a vztahy mezi nimi (relationship, de facto hrany/edge). Jsou užitečné pro práci s komplikovanými vztahy mezi entitami, které by se u klasických relačních databází musely řešit x-násobnými joiny. Jednou z nejznámějších databází tohoto typu je Neo4j. Toto povídání je zaměřeno na zacházení s její desktopovou verzí, která se dá stáhnout [zde](https://neo4j.com/download/).

## Spuštění
Po naběhnutí Neo4j vlezme nejprve do nastavení (ozubené kolo v levém panelu) do sekce "Data path". Zde se v textovém poli nalézá "root" projektů v Neo4j. To jo pro nás bude důležité zejména v okamžiku, kdy do Neo4j začneme nalévat data z externích souborů. Ty totiž budou muset být někde v tomto adresáři (resp. jeho poadresářích). Důvodem je securita (aby Neo4j nevidělo někam, kam nemá). Pozor - pokud tento adresář změníme, některé projekty nebudou fungovat (asi mají v nastavení starou cestu, která se musí ručně přepsat).  
![settings_data_path](neo4j_figures/settings_data_path.png)
Vyrobme nyní nový projekt (ikona adresáře nalevo nahoře) a pro přehlednost ho rovnou přejmenujme z defaultního "Project" na něco rozumnějšího (přejmenování realizujeme najetím na název a kliknutím na objevivší se ikonu). Přidejme do projektu databázi (tlačítko "Add" -> "Local DBMS"). Zde zdůrazněme, že se opravdu jedná o databázi a ne databázový kontejner. Tj. nejde přistupovat v rámci jedné databáze do databáze druhé - vlastně ani není možné, aby najednou běžely dvě databáze současně. Když totiž spustíme druhou, první se automaticky zavře.
![new_project](neo4j_figures/new_project.png)
Dovnitř databáze se dostaneme kliknutím na ní a poté na modré tlačítko "Open".
![info_database](neo4j_figures/into_database.png)
Pozn: pokud vám spuštění databáze zfailovalo, zkontrolujte, zda něco (např Jupyter notebook) náhodou neblokuje porty.

## Základní příkazy
Modus operandi práce v právě otevřeném Neo4j browseru spočívá v tom, že člověk napíše kód do určité buňky a obsah buňky potom spustí pomocí modrého trojúhelníku. Princip buněk je podobný jako u Jupyter notebooku - lze je spouštět napřeskáčku, přičemž výsledek práce jedné buňky může ovlivnit buňky ostatní. Oproti Jupyteru tu ale můžeme pozorovat jeden rozdíl - buňka se chová jako transakce. Tj. pokud běh buňky spadne na třetím příkazu, výsledky prvních dvou příkazů se revertují. Zdůrazněme nakonec, že použitý jazyk, ve kterém se tyto příkazy píší, už není SQL, nýbrž Cypher.  
Zobrazme si nejprve všechny nody v databázi. To provedeme pomocí
```
match(n) return n
```
Match říká, jaký pattern v databázi hledáme a return pak výsledke matche vrátí do browseru. Písmeno n sice značí node, ale to je jen pomůcka pro čtenáře - klidně by tu místo "n" mohlo být "tralala" a výsledek by byl úplně stejný. Pravda, v současnosti výsledkem jedno velké nic. Zkusme to napravit - vytvořme node pomocí
```
create (n)
```
Když nyní znovu spustíme "matchovací" příkaz, už nějaký výstup uvidíme. Panelem v levé části buňky lze překlikávat mezi jednotlivými reprezentacemi výstupu, vizuálně nejzajímavější bude ale asi ta první - graf.
![match_anonymous_node](neo4j_figures/match_anonymous_node.png)
Když nyní znovu spustíme "create (n)", neobjeví se žádný error ani warning - zkrátka se vytvoří nový node identický s tím předchozím. Když se obou nodů budeme chtít zbavit, použijeme
```
match(n) delete (n)
```
Vytváření anonymních nodů by moc užitečné nebylo. Když chceme použít node určitého typu, realizujeme to pomocí dvojtečky a názvu typu nodu (tzv. labelu), tj. takto:
```
create (n:Žirafa)
```
Z toho mimo jiné vidíme, že Neo4j úspěšně popasuje s českými znaky. Samozřejmě je otázkou, zda je i tak dobrý nápad české znaky používat - spíš ne.  
Výše ukázaným postupem jsme vytvořili node určitého typu. Počítá se s tím, že typ bude sdílet hromada nodů. Když potřebujeme do nodu přidat informaci, která bude vlastní jen nodu jednomu, použijeme tzv. property (vlastnosti). Ty se vytvoří pomocí složených závorek:
```
create (n:Žirafa{name:'Bětka'})
```
Properties je možné mít v jednom nodu více - při jejich zápisu je oddělíme čárkami. Co se týče uvozovek označující začátek a konec textových řetězců, ty mohou být jak jednoduché, tak dvojité:
```
create (n:Žirafa{name:"Dlouhokrčka", height_in_m:5.2})
```
Vytvořme pár nodů jiného typu. Z následujícího je vidět, že v jedné buňce browseru může být naráz více příkazů. jen musí mít odlišnou proměnnou označující node.
```
create (n1:Jídlo{name:"Jablko"})
create (n2:Jídlo{name:"Listy akácií"})
create (n3:Jídlo{name:"Hamburger"})
```
Na nody se můžeme podívat pomocí již zmíněného "match (n) return n".
![giraffe_1](neo4j_figures/giraffe_1.png)
Pokud bychom měli v databázi přiliš mnoho nodů a chtěli zobrazit jen pár z nich, abychom vlastně pochopili, jak vypadají, můžeme použít klíčové slovo limit:
```
match(n) return n limit 3
```
Vytvořme nyní mezi některými nody vztahy. To se udělá následujícím způsobem (pokud příkaz nekopírujete, ale ručně píšete, tak odřádkování dosáhnete pomocí SHIFT+ENTER):
```
match(z:Žirafa), (j:Jídlo)
where z.name = 'Bětka' and j.name = 'Jablko'
create (z)-[rel:má_ráda]->(j)
``` 
Co se tady vlastně děje? Na prvním řádku říkáme, že chceme nody typu Žirafa a Jídlo. Na řádku druhém klademe na nody určité podmínky - využíváme vlastností nodů, ke kterým přistupujeme prostřednictvím tečkové notace. Na řádku třetím vytváříme samotný vztah mezi nody. Ten je definovám v hranatých závorkách, přičemž "rel" je zastupující proměnná a "má_ráda" jméno vztahu. Pro označení nodů, mezi kterými vztah platí, použijeme kulaté závorky a jejich zastupující proměnnou. Pomlčka a šipka pak určují, kterým směrem vztah jde.  
Výstupem podmínky může být pochopitelně i více než pouze jeden node od každého typu:
```
match(z:Žirafa), (j:Jídlo)
where j.name = 'Listy akácií'
create (z)-[rel:má_ráda]->(j)
```
Vztahy mohou platit i mezi nody stejného typu:
```
match(z1:Žirafa), (z2:Žirafa)
where z1.name = 'Bětka' and z2.name = 'Dlouhokrčka'
create (z1)-[rel:je_matkou]->(z2)
```
![giraffe_rel](neo4j_figures/giraffe_rel.png)
Položme nyní databázi pár dotazů. Nejprve bychom chtěli znat všechny žirafy, které mají rády jablka.
```
match (z:Žirafa)-[:má_ráda]->(:Jídlo {name:"Jablko"}) return z
```
Všimněte si, že pro nody, které dále nebudeme potřebovat, nemusíme ani psát zastupující proměnnou.
![giraffe_likes_apple](neo4j_figures/giraffe_likes_apple.png)
Naopak když bychom chtěli znát všechna jídla, která má žirafa Bětka ráda, napíšeme
```
match (:Žirafa {name:"Bětka"})-[:má_ráda]->(j:Jídlo) return j
```
![giraffe_likes_apple_2](neo4j_figures/giraffe_likes_apple_2.png)
Na závěr kapitoly databázi promažeme. Když vyzkoušíme výše uvedené "match(n) delete (n)", vrátí se nám chyba "Neo.ClientError.Schema.ConstraintValidationFailed" s popiskem "Cannot delete node<0>, because it still has relationships. To delete this node, you must first delete its relationships.". Jelikož jsme ale líní a nechceme nejprve smazat vztahy a až potom nody, použijeme
```
match(n) detach delete (n)
``` 

## Načítání dat z csv souboru
Zkusme do Neo4j nalít data ze souboru animals.csv, který najdete v tomto repozitíři ve složce "tutorial_data". Naivním přístupem by bylo vložení nášeho souboru do "Data path" adresáře uvedeného v nastavení a posléze spuštění buňky obsahující
```
load csv with headers from
'file:///animals.csv' as csv
FIELDTERMINATOR ';'
create (anim:Animal {animal:csv.animal, eats:csv.food})
```
Pak bychom dostali chybovou hlášku typu "Neo.ClientError.Statement.ExternalResourceFailed" obsahujíc zprávu "Couldn't load the external resource at: file:/C:/vs/programy/poznamky_neo4j/relate-data/dbmss/dbms-bfd104f5-3c9c-474e-bc61-e99686446da7/import/animals.csv". Zde "C:/vs/programy/poznamky_neo4j" je můj "Data path" adresář. Nezbývá než soubor vložit do adresáře, kde ho Neo4j chce mít.  
Co se ale v příkazu vlastně děje? Na prvním řádku říkáme, že chceme načíst csvčko s hlavičkou. Druhý řádek prozrazuje, odkud a do jaké proměnné se má obsah souboru uložit. Třetí řádek stanovuje separátor polí - pokud je jím v souboru čárka, tak se fieldterminator udávat nemusí. Nakonec čtvrtý řádek říká, že se pro každý řádek z csv souboru má vytvořit node Animal, který má mít jednak parametr animal načítaný ze sloupce animal, jednak parametr eats načítaný ze sloupce food.  
Když to uděláme, dost možná se nám zobrazí pět nodů nadepsaných jídlem. Jenže my bychom chtěli mít nody nadepsané zvířetem. Pro dosažení tohoto cíle musíme kliknout na zobáček (<) v pravého horním rohu buňky, následně kliknout na chtěný typ nodu (zde Animal) a poté vybrat na řádku "Caption" chtěný parametr.  
![bad_top_property](neo4j_figures/bad_top_property.png)
Jenže je zde další problém. Ve vstupních datech byl jeden řádek (cow) zdvojen. Naštěstí zde pro opravu nemusíme dělat žádný preprocessing. Stačí když spustíme (po promazání databáze) načítací příkaz, kde "create" zaměníme za "merge".
```
load csv with headers from
'file:///animals.csv' as csv
FIELDTERMINATOR ';'
merge (anim:Animal {animal:csv.animal, eats:csv.food})
```
Všimněme si, že odstraněn byl jen zcela duplicitní řádek. Žirafy máme pořád dvě, protože každá jí něco jiného. Situace je ale komplikovanější. Merge se totiž fakticky pokusí o match a až když s tím neuspěje, tak vytvoří nový node. Co to fakticky znamená? Kdybychom měli 
```
create (pac1:Balíček {name:"Koala", version:1})
create (pac2:Balíček {name:"Koala"})
```
vytvořily by se nám dva nody, z nichž jeden by měl a jeden by neměl parametr version. Když ale použijeme kód
```
create (pac1:Balíček {name:"Koala", version:1})
create (pac2:Balíček {name:"Koala"})
```
vznikne jenom jeden node, přičemž parametr version bude mít. Na prvním řádku se totiž začne hledat balíček o daném jménu a verzi, nebude nalezen a tak se založí. Na řádku druhém se bude hledat balíček o daném jméně (verze je nyní irrelevantní), nalezen bude a tak se druhý node už nevytvoří.  
K čemu je to dobré? Za chvíli načteme pomocí příkazu 
```
load csv with headers from
"file:///pandas_required_packages.csv" as csv
FIELDTERMINATOR ';'
merge (pac1:Package {name:csv.name, version:csv.version, license:csv.license, summary:csv.summary})
merge (pac2:Package {name:csv.required_packages})
create (pac1)-[spoj:REQUIRES]->(pac2)
```
data ze souboru pandas_required_packages.csv (k nalezení stejně jako animals.csv ve složce tutorial_data). Tento soubor má strukturu typu
```
name;version;license;summary;required_packages
pandas;1.3.5;BSD-3-Clause;Powerful data structures for data analysis, time series, and statistics;python-dateutil
pandas;1.3.5;BSD-3-Clause;Powerful data structures for data analysis, time series, and statistics;hypothesis
pandas;1.3.5;BSD-3-Clause;Powerful data structures for data analysis, time series, and statistics;pytz
hypothesis;6.31.4;MPL v2;A library for property-based testing;tzdata
hypothesis;6.31.4;MPL v2;A library for property-based testing;pytz
``` 
Pokud bychom měli namísto merge create, vedl by řádek s "(pac2:Package {name:csv.required_packages})" u příkladu dat k vytvoření dodatečného nodu, který bychom nechtěli.  
Potížím ale ještě není konec. Některé balíčky totiž žádné prerekvizitní balíčky nepožadují, tj. v poli required_packages nemají nic. Toto "nic" neo4j interpretuje jako null hodotu. Jenomže merge s null hodnotami nedokáže pracovat. Proto jsem musel na takováto místa vložit neexistující balíček "fake_package". Marge tak úspěšně doběhne, ale v databází nyní máme nesmyslný node. Ten vymažeme pomocí
```
match (pac3:Package {name:"fake_package"}) detach delete pac3
```